## pix2pix(U-Net + GAN) experiments

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Model definitions

In [2]:
from keras import objectives
from keras import backend as K
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Input, Merge, merge
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, Deconvolution2D
from keras.layers.core import Activation, Dropout

Using TensorFlow backend.


In [3]:
# U-Net Generator
def g_unet(nf, model_name, in_ch=1, out_ch=1, batch_size=1, alpha=0.2):
    ''' параметры:
    input shape = (100, 100, in_ch)
    output = (100, 250, out_ch)
    nf - число фильтров на входном слое
    alpha - параметр LeakyReLU
    '''
    i = Input(shape=(100, 100, in_ch))
    # (100, 100, in_ch)
    
    conv1 = Convolution2D(nf, 6, 6, border_mode='same', subsample=(5, 5))(i)
    conv1 = BatchNormalization(mode=2, axis=1)(conv1)
    x = LeakyReLU(alpha)(conv1)
    # (20, 20, nf)
    
    conv2 = Convolution2D(nf*5, 6, 6, border_mode='same', subsample=(5, 5))(x)
    conv2 = BatchNormalization(mode=2, axis=1)(conv2)
    x = LeakyReLU(alpha)(conv2)
    # (4, 4, nf*5)
    
    conv3 = Convolution2D(nf*10, 3, 3, border_mode='same', subsample=(2, 2))(x)
    conv3 = BatchNormalization(mode=2, axis=1)(conv3)
    x = LeakyReLU(alpha)(conv3)
    # (2, 2, nf*10)

    conv4 = Convolution2D(nf*10, 2, 2, border_mode='valid', subsample=(1, 1))(x)
    conv4 = BatchNormalization(mode=2, axis=1)(conv4)
    x = LeakyReLU(alpha)(conv4)
    # (1, 1, nf*10)

    dconv1 = Deconvolution2D(nf*10, 2, 2, output_shape=(batch_size, 2, 2, nf*10), subsample=(1, 1))(x)
    dconv1 = BatchNormalization(mode=2, axis=1)(dconv1)
    dconv1 = Dropout(0.5)(dconv1)
    x = merge([dconv1, conv3], mode='concat', concat_axis=3)
    x = LeakyReLU(alpha)(x)
    # (2, 2, nf*(10 + 10))

    dconv2 = Deconvolution2D(nf*5, 2, 2, output_shape=(batch_size, 4, 4, nf*5), subsample=(2, 2))(x)
    dconv2 = BatchNormalization(mode=2, axis=1)(dconv2)
    x = merge([dconv2, conv2], mode='concat', concat_axis=3)
    x = LeakyReLU(alpha)(x)
    # (4, 4, nf*(5 + 5))

    dconv3 = Deconvolution2D(nf, 2, 2, output_shape=(batch_size, 20, 20, nf), subsample=(2, 2))(x)
    dconv3 = BatchNormalization(mode=2, axis=1)(dconv3)
    x = merge([dconv3, conv1], mode='concat', concat_axis=3)
    x = LeakyReLU(alpha)(x)
    # (20, 20, nf*(1 + 1))

    dconv4 = Deconvolution2D(1, 2, 2, output_shape=(batch_size, 250, 100, out_ch))(x)
    # (250, 100, out_ch)

    out = Activation('tanh')(dconv4)
    unet = Model(i, out, name=model_name)
    
    return unet

In [12]:
# Discriminator
def discriminator(nf, a_ch=1, b_ch=1, opt=Adam(lr=2e-4, beta_1=0.5), alpha=0.2, model_name='d'):
    ''' параметры:
    a_ch - число каналов первого изображения
    b_ch - число каналов второго
    nf - число фильтров на входном слое
    alpha - параметр LeakyReLU
    '''
    i = Input(shape=(500, 100, a_ch + b_ch))
    # (500, 100, a_ch + b_ch)
    
    conv1 = Convolution2D(nf, 6, 6, border_mode='same', subsample=(5,5))(i)
    x = LeakyReLU(alpha)(conv1)
    # (100, 20, nf)
    
    conv2 = Convolution2D(nf*5, 6, 6, border_mode='same', subsample=(5,5))(x)
    x = LeakyReLU(alpha)(conv2)
    # (20, 4, nf*5)
    
    conv3 = Convolution2D(1, 3, 3, border_mode='same', subsample=(2,2))(x)
    out = Activation('sigmoid')(conv3)
    # (10, 2, 1)
    
    d = Model(i, out, name=model_name)
    
    def d_loss(y_true, y_pred):
        L = objectives.binary_crossentropy(K.batch_flatten(y_true), K.batch_flatten(y_pred))
        return L
    
    d.compile(optimizer=opt, loss=d_loss)
    return d

In [25]:
def full_generator(nf, in_ch=1, out_ch=1, batch_size=1, alpha=0.2, model_name='f_gen'):
    a1 = Input(shape=(100, 100, in_ch))
    a2 = Input(shape=(100, 100, in_ch))
    gen1 = g_unet(nf, 'unet1', in_ch, out_ch, batch_size, alpha)
    out1 = gen1(a1)
    gen2 = g_unet(nf, 'unet2', in_ch, out_ch, batch_size, alpha)
    out2 = gen2(a2)
    out = merge([out1, out2], mode='concat', concat_axis=1)
    f_gen = Model([a1, a2], out, name=model_name)
    return f_gen

In [23]:
def pix2pix(atob, d, a_ch=1, b_ch=1, alpha=100, opt=Adam(lr=2e-4, beta_1=0.5), model_name='pix2pix'):
    '''
    atob - full generator
    d - discriminator
    '''
    a1 = Input(shape=(100, 100, a_ch))
    a2 = Input(shape=(100, 100, a_ch))
    b = Input(shape=(500, 100, b_ch))
    
    # генерируем картинку на основе a1 и a2 с помощью объединенного генератора:
    bp = atob([a1, a2])
    
    # дискриминатор получает на вход пару изображений
    d_in = merge([b, bp], mode='concat', concat_axis=3)
    pix2pix = Model([a1, a2, b], d(d_in), name=model_name)
    
    def p2p_loss(y_true, y_pred):
        y_true_flat = K.batch_flatten(y_true)
        y_pred_flat = K.batch_flatten(y_pred)
        
        # adversarial loss
        L_adv = objectives.binary_crossentropy(y_true_flat, y_pred_flat)
        
        # atob loss
        b_flat = K.batch_flatten(b)
        bp_flat = K.batch_flatten(bp)
        L_atob = K.mean(K.abs(b_flat - bp_flat))
        
        return L_adv + alpha*L_atob
    
    # обучаем генератор - фризим дискриминатор
    pix2pix.get_layer('d').trainable = False
    
    pix2pix.compile(optimizer=opt, loss=p2p_loss)
    return pix2pix

### Dataset loading + preprocessing

In [ ]:
from PIL import Image

In [ ]:
H = 100
W = 500
dataPrefix = '../data/sand/full_dataset/trend1/panorama/sample'
dataExt = '.jpg'

def loadImage(i):
    fileName = dataPrefix + str(i) + dataExt
    im = Image.open(fileName)
    return np.array(im)

In [ ]:
im = loadImage(1)

In [ ]:
im.shape

In [ ]:
im = im.reshape(1, 100, 100, 1)
y = np.array([0])

### Training

In [26]:
f_gen = full_generator(5)
d = discriminator(5)
p2p = pix2pix(f_gen, d)

In [16]:
f_gen.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 100, 100, 1)   0                                            
____________________________________________________________________________________________________
convolution2d_12 (Convolution2D) (None, 20, 20, 5)     185                                          
____________________________________________________________________________________________________
batchnormalization_15 (BatchNorm (None, 20, 20, 5)     80                                           
____________________________________________________________________________________________________
leakyrelu_18 (LeakyReLU)         (None, 20, 20, 5)     0                                            
___________________________________________________________________________________________

In [35]:
d.output_shape

(None, 10, 2, 1)

In [27]:
p2p.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_19 (InputLayer)            (None, 100, 100, 1)   0                                            
____________________________________________________________________________________________________
input_20 (InputLayer)            (None, 100, 100, 1)   0                                            
____________________________________________________________________________________________________
input_21 (InputLayer)            (None, 500, 100, 1)   0                                            
____________________________________________________________________________________________________
f_gen (Model)                    (None, 500, 100, 1)   94786       input_19[0][0]                   
                                                                   input_20[0][0]          

### Using trained NN

In [33]:
p2p.layers

In [ ]:
f_gen.output_shape

In [ ]:
d.input_shape

In [ ]:
d.output_shape

In [ ]:
d.name

In [ ]:
model = Sequential(name='00')

In [ ]:
model.name